### We will be using pytorch datasets and data loaders to implement u-net residual model 

In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, sampler,Subset



import logging
import numpy as np
from utils import *


import time
import copy


#### Step1 : Create dataset and dataloader  
#### Creating the dataset from Imagefolder having folder structure like
--Image-folder

    --0 
    
    --1 
    
    --2
    
    .
    
    .
    
    --5
    
    

In [105]:
root_dir = '/home/abharani/cs231n_project'
data_dir =  '/home/abharani/data'
# csv_path = os.path.join(data_dir,'train.csv')
# small_csv_path = os.path.join(data_dir,'small_train.csv')
# tiny_csv_path = os.path.join(data_dir,'tiny_train.csv')
dir_checkpoint = os.path.join(root_dir, 'checkpoints')

import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader


def train_val_dataset(dataset, val_split=0.25, generate_small=False):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    if generate_small:
        print("Generating Small Train/Valid Dataset")
        datasets['train'] = Subset(dataset, train_idx[0:200])
        datasets['val'] = Subset(dataset, val_idx[0:50])
    else:
        datasets['train'] = Subset(dataset, train_idx)
        datasets['val'] = Subset(dataset, val_idx)        
    return datasets

dataset = ImageFolder(os.path.join(data_dir,'output'), transform=Compose([Resize((224,224)),ToTensor(), 
                                                                      Normalize(mean=[0.485, 0.456, 0.406],
                                                                               std=[0.229, 0.224, 0.225]) 
                                                                      ]))
print(len(dataset))
datasets = train_val_dataset(dataset,generate_small=False)
print(len(datasets['train']))
print(len(datasets['val']))
# The original dataset is available in the Subset class
print(datasets['train'].dataset)


x,y = datasets['train'][0]
print(x.shape, y)

10615
7961
2654
Dataset ImageFolder
    Number of datapoints: 10615
    Root location: /home/abharani/data/output
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize: (TensorImage,object) -> encodes (TensorImage,object) -> decodes
           )
torch.Size([3, 224, 224]) 5


In [3]:
# def load_data(root_path, batch_size, phase):
#     transform_dict = {
#         'train': transforms.Compose(
#         [transforms.RandomResizedCrop(224),
#          transforms.RandomHorizontalFlip(),
#          transforms.ToTensor(),
#          transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                               std=[0.229, 0.224, 0.225]),
#          ]),
#         'valid': transforms.Compose(
#         [transforms.Resize(224),
#          transforms.ToTensor(),
#          transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                               std=[0.229, 0.224, 0.225]),
#          ])}
#     data = datasets.ImageFolder(root=root_path, transform=transform_dict[phase])
#     print(len(data))
#     data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=4)
#     return data_loader 

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:

base_model = torchvision.models.resnet18(pretrained=False)

base_model = base_model.to(device)

# list(base_model.children())



In [6]:
def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )

class ResNetUNet(nn.Module):

    def __init__(self, n_class):
        super().__init__()
        
        self.base_model = torchvision.models.resnet18(pretrained=True)

        self.base_layers = list(base_model.children())                
        
        self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)        
        self.layer1_1x1 = convrelu(64, 64, 1, 0)       
        self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)        
        self.layer2_1x1 = convrelu(128, 128, 1, 0)  
        self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)        
        self.layer3_1x1 = convrelu(256, 256, 1, 0)  
        self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
        self.layer4_1x1 = convrelu(512, 512, 1, 0)  
        
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
        self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
        self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
        self.conv_up0 = convrelu(64 + 256, 128, 3, 1)
        
        self.conv_original_size0 = convrelu(3, 64, 3, 1)
        self.conv_original_size1 = convrelu(64, 64, 3, 1)
        self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)
        
        self.conv_last = nn.Conv2d(64, n_class, 1)
        # MY experiment
        self.fc = nn.Linear(301056, 6)
        
    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)
        
        layer0 = self.layer0(input)            
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)        
        layer4 = self.layer4(layer3)
        
        layer4 = self.layer4_1x1(layer4)
        x = self.upsample(layer4)
        layer3 = self.layer3_1x1(layer3)
        x = torch.cat([x, layer3], dim=1)
        x = self.conv_up3(x)
 
        x = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x = torch.cat([x, layer2], dim=1)
        x = self.conv_up2(x)

        x = self.upsample(x)
        layer1 = self.layer1_1x1(layer1)
        x = torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x)

        x = self.upsample(x)
        layer0 = self.layer0_1x1(layer0)
        x = torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x)
        
        x = self.upsample(x)
        x = torch.cat([x, x_original], dim=1)
        x = self.conv_original_size2(x)        
        
        out = self.conv_last(x)        

        # MY experiment
        x = out.view(out.size(0), -1)
        x = self.fc(x)
        
        # x = F.adaptive_avg_pool2d(out, output_size=1)
        # print(x.size(),x)

        
        return x



In [7]:
def eval_model(model, val_loader, device):
    """Evaluation without the densecrf with the dice coefficient"""
    model.eval()

    n_val = len(val_loader)  # the number of batch
    tot = 0

    for batch in val_loader:
        imgs, true_label = batch[0], batch[1]
        imgs = imgs.to(device=device, dtype=torch.float32)
        true_label = true_label.to(device=device, dtype=torch.long)
        with torch.no_grad():
            pred_label = model(imgs)

        tot += F.cross_entropy(pred_label, true_label).item()

    model.train()
    return tot / n_val

## Use new_train_model below for 224x224 compressed images inside output folder

In [10]:
def new_train_model(model, device, epochs=5, batch_size=32, lr=0.001, val_percent=0.1, save_cp=True):

    dataloaders = {x:DataLoader(datasets[x],batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train','val']}
    train_loader = dataloaders['train']
    val_loader = dataloaders['val']

#     x,y = next(iter(dataloaders['train']))
#     print(x.shape, y.shape)

    dataset_sizes = {x: len(datasets[x]) for x in datasets.keys()}
    
    
    
    # dataset = BasicDataset(csv_file=tiny_csv_path, data_dir=data_dir)
    # n_val = int(len(dataset) * val_percent)
    # n_train = len(dataset) - n_val
    # train, val = random_split(dataset, [n_train, n_val])
    # train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    # val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)
    # dataloaders = {'train': train_loader, 'val':val_loader}
    # dataset_sizes = {'train':n_train, 'val':n_val}

    # default `log_dir` is "runs" - we'll be more specific here
    writer = SummaryWriter('runs/experiment_2_perfect')
    global_step = 0

    # print(f'''Starting training:
    #     Epochs:          {epochs}
    #     Batch size:      {batch_size}
    #     Learning rate:   {lr}
    #     Training size:   {n_train}
    #     Validation size: {n_val}
    #     Checkpoints:     {save_cp}
    #     Device:          {device.type}
    # ''')

    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0


    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))

        model.train()

        epoch_loss = 0.00 
        epoch_accuracy = 0.00
        running_loss = 0.00
        running_corrects = 0.00 
        for i, batch in enumerate(train_loader) :
            
#             print('Epoch {}, Step{}'.format(epoch, i))
            
            imgs = batch[0].to(device, dtype=torch.float32)
            # imgs = torch.reshape(imgs,(imgs.shape[0],imgs.shape[3],imgs.shape[1],imgs.shape[2])) #  inputs.reshape [N, C, W, H]

            true_label = batch[1].to(device=device, dtype=torch.long)

            logits = model(imgs)
            _, pred_label = torch.max(logits, 1)
            loss = criterion(logits, true_label)

            epoch_loss += loss.item()
            running_loss += loss.item() * imgs.size(0) # batch_size 
            running_corrects += torch.sum(pred_label == true_label.data)

            #tensorboard
            img_grid = torchvision.utils.make_grid(imgs)
            # writer.add_image("training Images grid" , img_grid)
            writer.add_scalar('Training Loss', loss.item(), global_step)

            if i %10 ==0: # print loss every 10th step
                print('Epoch {} , Train Step {}, Training loss: {}'.format(epoch, i , loss.item()))

            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_value_(model.parameters(), 0.1)
            optimizer.step()


            global_step += 1
#             if global_step % (len(dataset) // (10 * batch_size)) == 0:
                # for tag, value in model.named_parameters():
                #     tag = tag.replace('.', '/')
                #     writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                #     writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)
        val_score = eval_model(model, val_loader, device)
        scheduler.step()
        print('Validation loss: {}'.format(val_score))

        #tensorboard
        writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)
        writer.add_scalar('Validation Loss', val_score, global_step)
        writer.add_images('images', imgs, global_step)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(' epoch {}, Loss: {:.4f} Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))
        writer.add_scalar("epoch loss", epoch_loss, epoch)
        writer.add_scalar("epoch acc", epoch_acc, epoch)

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                print('Created checkpoint directory')
            except OSError:
                pass
            torch.save(model.state_dict(),
                       dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
            print(f'Checkpoint {epoch + 1} saved !')
    writer.close()

In [11]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# device='cpu'
num_class = 6

model = ResNetUNet(num_class).to(device)

# freeze backbone layers
# Comment out to finetune further
for l in model.base_layers:
    for param in l.parameters():
        param.requires_grad = False

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)        
        
# model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=1)

model = new_train_model(model, device, epochs=5, batch_size=32, lr=0.001,val_percent=0.1, save_cp=True)

cuda
Epoch 0/4
Epoch 0 , Train Step 0, Training loss: 1.7884879112243652
Epoch 0 , Train Step 10, Training loss: 1.7266604900360107
Epoch 0 , Train Step 20, Training loss: 1.8178170919418335
Epoch 0 , Train Step 30, Training loss: 1.704101800918579
Epoch 0 , Train Step 40, Training loss: 1.644623875617981
Epoch 0 , Train Step 50, Training loss: 1.5260205268859863
Epoch 0 , Train Step 60, Training loss: 1.713887333869934
Epoch 0 , Train Step 70, Training loss: 1.8749359846115112
Epoch 0 , Train Step 80, Training loss: 1.4829652309417725
Epoch 0 , Train Step 90, Training loss: 1.7189066410064697
Epoch 0 , Train Step 100, Training loss: 1.5081465244293213
Epoch 0 , Train Step 110, Training loss: 1.6298831701278687
Epoch 0 , Train Step 120, Training loss: 1.7084238529205322
Epoch 0 , Train Step 130, Training loss: 1.798529028892517
Epoch 0 , Train Step 140, Training loss: 1.6502529382705688
Epoch 0 , Train Step 150, Training loss: 1.5643610954284668
Epoch 0 , Train Step 160, Training loss:

### End of Training

In [ ]:
for i, batch in enumerate(train_loader) :
    print(batch[0].shape, batch[1].shape)

In [57]:
from torchvision.transforms import Compose, ToTensor, Resize, ToPILImage
from PIL import Image

In [89]:
import os 
from os.path import splitext
from os import listdir
import numpy as np
import pandas as pd
from pandas import DataFrame
import os 
import skimage 
from skimage import io 
import torch
from torch.utils.data import Dataset
import logging
from PIL import Image


class BasicDataset(Dataset):
    def __init__(self, csv_file, data_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """        
        self.data_frame = pd.read_csv(csv_file)

        ## Local images only  
        
        # local_files_df = DataFrame([os.path.splitext(filename)[0] for filename in os.listdir('/Users/abharani/Documents/myworkspace/cs231n_project/data/train_images/') if filename.endswith(".tiff")], columns=["image_id"])
        # self.data_frame = local_files_df.join(data_frame.set_index('image_id'), on='image_id')
        # ##

        self.data_dir = data_dir
        self.transform = transform
        logging.info('Creating dataset with {} examples'.format(len(self.data_frame)))
        
    def __len__(self):
        return len(self.data_frame)

    @classmethod
    def preprocess(cls, image):
        WINDOW_SIZE = 128
        STRIDE = 64
        K = 16
        
        image, best_coordinates, best_regions = generate_patches(image, window_size=WINDOW_SIZE, stride=STRIDE, k=K)
        glued_image = glue_to_one_picture(best_regions, window_size=WINDOW_SIZE, k=K)
        
        return glued_image


    def __getitem__(self, idx):
        
        image_id = self.data_frame.iloc[idx]['image_id']
        image_file_path = os.path.join(os.path.join(self.data_dir,'train_images'),image_id + ".tiff")
        image = skimage.io.MultiImage(image_file_path)[-1]
        image = np.array(image)

        isup_grade = self.data_frame.iloc[idx]['isup_grade']

        image = self.preprocess(image)
        PIL_image = Image.fromarray((image * 255).astype(np.uint8))
#         print("Image type {}".format(image.type)) #numpy.ndarray
#         print(PIL_image.type)                     # Image object      

        
        if self.transform:
            
            image = self.transform(PIL_image)
        
        return {'image': image, 'isup_grade' : isup_grade}




In [90]:
my_transform=Compose([Resize((224,224)),ToTensor(), Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) 
                                                                      ])

In [91]:
dataset = BasicDataset(csv_file='/project/data/train.csv', data_dir='/project/data',transform=my_transform)

len(dataset)

10616

In [92]:
for i in range(len(datasets['train'])):
    sample = dataset[i]

    print(i, sample['image'].shape, sample['isup_grade'])

    if i == 3:
        plt.show()
        break

0 torch.Size([3, 224, 224]) 0
1 torch.Size([3, 224, 224]) 0
2 torch.Size([3, 224, 224]) 4
3 torch.Size([3, 224, 224]) 4


In [93]:
def train_val_dataset(dataset, val_split=0.25, generate_small=False):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    if generate_small:
        print("Generating Small Train/Valid Dataset")
        datasets['train'] = Subset(dataset, train_idx[0:200])
        datasets['val'] = Subset(dataset, val_idx[0:50])
    else:
        datasets['train'] = Subset(dataset, train_idx)
        datasets['val'] = Subset(dataset, val_idx)        
    return datasets

# dataset = ImageFolder(os.path.join(data_dir,'output'), transform=Compose([Resize((224,224)),ToTensor(), 
#                                                                       Normalize(mean=[0.485, 0.456, 0.406],
#                                                                                std=[0.229, 0.224, 0.225]) 
#                                                                       ]))
print(len(dataset))
datasets = train_val_dataset(dataset,generate_small=True)
print(len(datasets['train']))
print(len(datasets['val']))
# The original dataset is available in the Subset class
print(datasets['train'])

10616
Generating Small Train/Valid Dataset
200
50


In [56]:
def compute_statistics(image):
    """
    Args:
        image                  numpy.array   multi-dimensional array of the form WxHxC
    
    Returns:
        ratio_white_pixels     float         ratio of white pixels over total pixels in the image 
    """
    width, height = image.shape[0], image.shape[1]
    num_pixels = width * height
    
    num_white_pixels = 0
    
    summed_matrix = np.sum(image, axis=-1)
    # Note: A 3-channel white pixel has RGB (255, 255, 255)
    num_white_pixels = np.count_nonzero(summed_matrix > 620)
    ratio_white_pixels = num_white_pixels / num_pixels
    
    green_concentration = np.mean(image[1])
    blue_concentration = np.mean(image[2])
    
    return ratio_white_pixels, green_concentration, blue_concentration

def select_k_best_regions(regions, k=20):
    """
    Args:
        regions               list           list of 2-component tuples first component the region, 
                                             second component the ratio of white pixels
                                             
        k                     int            number of regions to select
    """
    regions = [x for x in regions if x[3] > 180 and x[4] > 180]
    k_best_regions = sorted(regions, key=lambda tup: tup[2])[:k]
    return k_best_regions

def generate_patches(image, window_size=200, stride=128, k=20):
    
#     image = skimage.io.MultiImage(slide_path)[-2]
#     image = np.array(image)
    
    max_width, max_height = image.shape[0], image.shape[1]
    regions_container = []
    i = 0
    
    while window_size + stride*i <= max_height:
        j = 0
        
        while window_size + stride*j <= max_width:            
            x_top_left_pixel = j * stride
            y_top_left_pixel = i * stride
            
            patch = image[
                x_top_left_pixel : x_top_left_pixel + window_size,
                y_top_left_pixel : y_top_left_pixel + window_size,
                :
            ]
            
            ratio_white_pixels, green_concentration, blue_concentration = compute_statistics(patch)
            
            region_tuple = (x_top_left_pixel, y_top_left_pixel, ratio_white_pixels, green_concentration, blue_concentration)
            regions_container.append(region_tuple)
            
            j += 1
        
        i += 1
    
    k_best_region_coordinates = select_k_best_regions(regions_container, k=k)
    k_best_regions = get_k_best_regions(k_best_region_coordinates, image, window_size)
    
    return image, k_best_region_coordinates, k_best_regions

def display_images(regions, title):
    fig, ax = plt.subplots(5, 4, figsize=(15, 15))
    
    for i, region in regions.items():
        ax[i//4, i%4].imshow(region)
    
    fig.suptitle(title)
    
    
def get_k_best_regions(coordinates, image, window_size=512):
    regions = {}
    for i, tup in enumerate(coordinates):
        x, y = tup[0], tup[1]
        regions[i] = image[x : x+window_size, y : y+window_size, :]
    
    return regions


def glue_to_one_picture(image_patches, window_size=200, k=16):
    side = int(np.sqrt(k))
    image = np.zeros((side*window_size, side*window_size, 3), dtype=np.int16)
        
    for i, patch in image_patches.items():
        x = i // side
        y = i % side
        image[
            x * window_size : (x+1) * window_size,
            y * window_size : (y+1) * window_size,
            :
        ] = patch
    
    return image

In [ ]:
torch.device

In [ ]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
from fastai2.vision.widgets import *
from fastai2.vision.all import *

In [ ]:
path = Path(data_dir)

In [ ]:
dls = bears.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=1,nrows=1)

In [ ]:
bears = bears.new(item_tfms=RandomResizedCrop(128, min_scale=0.3),batch_tfms=aug_transforms(mult=2))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, model, metrics=error_rate)
learn.fine_tune(4)